# Tugas Individu 3

## Aurellio Fishandy
### 1806199966

### Mengimport library yang digunakan

In [1]:
import numpy as np
import pandas as pd
from random import randrange

### Mempersiapkan data yang akan di proses

In [2]:
iris = pd.read_csv("D:\TUGAS\AI\Iris.csv")

In [3]:
X = iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
X = np.array(X)
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [4]:
dummies = pd.get_dummies(iris.Species)
merged_data = pd.concat([iris,dummies], axis=1)
Y = merged_data.drop(columns=['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species'])
Y = np.array(Y)
Y[:5]

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

### Membagi data menjadi train & test

In [5]:
def shuffle_split_data(X, y):
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, 70)

    X_train = X[split]
    y_train = y[split]
    X_test =  X[~split]
    y_test = y[~split]

    return X_train, y_train, X_test, y_test
X_train, Y_train, X_test, Y_test = shuffle_split_data(X, Y)

### Membuat fungsi Neural Network 

In [6]:
def NeuralNetwork1(X_train, Y_train, epochs=10, nodes=[], lr=0.10):
    hidden_layers = len(nodes) - 1
    weights = InitializeWeights(nodes)

    for epoch in range(1, epochs+1):
        weights = Train(X_train, Y_train, lr, weights)

        if(epoch % 20 == 0):
            print("Epoch {}".format(epoch))
            print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, weights)))
            
    return weights

In [7]:
def NeuralNetwork1(X_train, Y_train, epochs=10, nodes=[], lr=0.20):
    hidden_layers = len(nodes) - 1
    weights = InitializeWeights(nodes)

    for epoch in range(1, epochs+1):
        weights = Train(X_train, Y_train, lr, weights)

        if(epoch % 20 == 0):
            print("Epoch {}".format(epoch))
            print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, weights)))
            
    return weights

In [8]:
def NeuralNetwork1(X_train, Y_train, epochs=10, nodes=[], lr=0.30):
    hidden_layers = len(nodes) - 1
    weights = InitializeWeights(nodes)

    for epoch in range(1, epochs+1):
        weights = Train(X_train, Y_train, lr, weights)

        if(epoch % 20 == 0):
            print("Epoch {}".format(epoch))
            print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, weights)))
            
    return weights

In [9]:
def InitializeWeights(nodes):
    layers, weights = len(nodes), []
    
    for i in range(1, layers):
        w = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)]
              for j in range(nodes[i])]
        weights.append(np.matrix(w))
    
    return weights

### Fungsi FPNN

In [10]:
def ForwardPropagation(x, weights, layers):
    activations, layer_input = [x], x
    for j in range(layers):
        activation = Sigmoid(np.dot(layer_input, weights[j].T))
        activations.append(activation)
        layer_input = np.append(1, activation) # Augment with bias
    
    return activations

### Fungsi BPNN

In [11]:
def BackPropagation(y, activations, weights, layers):
    outputFinal = activations[-1]
    error = np.matrix(y - outputFinal) # Error at output
    
    for j in range(layers, 0, -1):
        currActivation = activations[j]
        
        if(j > 1):
            # Augment previous activation
            prevActivation = np.append(1, activations[j-1])
        else:
            # First hidden layer, prevActivation is input (without bias)
            prevActivation = activations[0]
        
        delta = np.multiply(error, SigmoidDerivative(currActivation))
        weights[j-1] += lr * np.multiply(delta.T, prevActivation)

        w = np.delete(weights[j-1], [0], axis=1) # Remove bias from weights
        error = np.dot(delta, w) # Calculate error for current layer
    
    return weights

### Memproses dan menetapkan weights yang didapatkan oleh BPNN

In [12]:
def Train(X, Y, lr, weights):
    layers = len(weights)
    for i in range(len(X)):
        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x)) # Augment feature vector
        
        activations = ForwardPropagation(x, weights, layers)
        weights = BackPropagation(y, activations, weights, layers)

    return weights

### Fungsi Sigmoid & SigmoidDerivative yang digunakan untuk memproses data 

In [13]:
def Sigmoid(x):
    return 1 / (1 + np.exp(-x))

def SigmoidDerivative(x):
    return np.multiply(x, 1-x)

### Fungsi untuk memproses data yang telah dipersiapkan

In [14]:
def Predict(item, weights):
    layers = len(weights)
    item = np.append(1, item) # Augment feature vector
    
    ##_Forward Propagation_##
    activations = ForwardPropagation(item, weights, layers)
    
    outputFinal = activations[-1].A1
    index = FindMaxActivation(outputFinal)

    # Initialize prediction vector to zeros
    y = [0 for i in range(len(outputFinal))]
    y[index] = 1  # Set guessed class to 1

    return y # Return prediction vector


def FindMaxActivation(output):
    """Find max activation in output"""
    m, index = output[0], 0
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    
    return index

In [15]:
def Accuracy(X, Y, weights):
    """Run set through network, find overall accuracy"""
    correct = 0

    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        guess = Predict(x, weights)

        if(y == guess):
            # Guessed correctly
            correct += 1

    return correct / len(X)

In [16]:
f = len(X[0]) # Number of features
o = len(Y[0]) # Number of outputs / classes
layers = [f, 5, 10, o] # Number of nodes in layers

### Melakukan pengetesan Neural Network dengan Learning Rate yang telah diatur
#### Hasil akurasi training 

In [27]:
lr, epochs = 0.10, 100

weights1 = NeuralNetwork1(X_train, Y_train, epochs=epochs, nodes=layers, lr=lr);

Epoch 20
Training Accuracy:0.6857142857142857
Epoch 40
Training Accuracy:0.6857142857142857
Epoch 60
Training Accuracy:0.7047619047619048
Epoch 80
Training Accuracy:0.8380952380952381
Epoch 100
Training Accuracy:0.8857142857142857


In [29]:
print("Akurasi: {}".format(Accuracy(X_test, Y_test, weights1)))
test1 = ("{}".format(Accuracy(X_test, Y_test, weights1)))
train1 = ("{}".format(Accuracy(X_train, Y_train, weights1)))

Akurasi: 0.8444444444444444


In [19]:
lr, epochs = 0.20, 100

weights2 = NeuralNetwork1(X_train, Y_train, epochs=epochs, nodes=layers, lr=lr);

Epoch 20
Training Accuracy:0.6857142857142857
Epoch 40
Training Accuracy:0.6857142857142857
Epoch 60
Training Accuracy:0.8095238095238095
Epoch 80
Training Accuracy:0.8761904761904762
Epoch 100
Training Accuracy:0.9047619047619048


In [20]:
print("Akurasi: {}".format(Accuracy(X_test, Y_test, weights)))
test2 = ("{}".format(Accuracy(X_test, Y_test, weights2)))
train2 = ("{}".format(Accuracy(X_train, Y_train, weights2)))

Akurasi: 0.8888888888888888


In [31]:
lr, epochs = 0.30, 100

weights3 = NeuralNetwork1(X_train, Y_train, epochs=epochs, nodes=layers, lr=lr);

Epoch 20
Training Accuracy:0.6857142857142857
Epoch 40
Training Accuracy:0.6857142857142857
Epoch 60
Training Accuracy:0.7714285714285715
Epoch 80
Training Accuracy:0.8857142857142857
Epoch 100
Training Accuracy:0.9238095238095239


In [32]:
print("Akurasi: {}".format(Accuracy(X_test, Y_test, weights)))
test3 = ("{}".format(Accuracy(X_test, Y_test, weights3)))
train3 = ("{}".format(Accuracy(X_train, Y_train, weights3)))

Akurasi: 0.6222222222222222


In [ ]:
hasil = pd.DataFrame(
     [[train1, test1],
     [train2, test2],
     [train3, test3]
    columns=["Training","Testing"],
    index=["Learing Rate 10%","Learing Rate 20%","Learing Rate 30%"]
)

hasil